# II. Clean Comments Column

# **Something like this to start**

In [43]:
reviews['comments'].head()

0    The apartment was as advertised and Frank was ...
1    It was a pleasure to stay at Frank's place. Th...
2    The apartment description is entirely faithful...
3    Thoroughly enjoyed my time at Frank's home. Ha...
4    Great value for the money! This location has e...
Name: comments, dtype: object

In [44]:
# the below code was inspired by lesson 5.06 on NLP Modeling

def remove_html(comment):
    '''
    function to remove html 
    and lowercase all text
    '''
    #lowercase
    comment = comment.lower()
    
    #remove html
    no_html = BeautifulSoup(comment).text
    
    return no_html

In [45]:
# Apply fuction to data
reviews['clean_comments'] = reviews['comments'].apply(remove_html)

/Users/petermurphy/opt/anaconda3/envs/dsi/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [46]:
print(reviews.shape)
reviews.head()

(228344, 7)


,listing_id,id,date,reviewer_id,reviewer_name,comments,clean_comments
0,3781,37776825,2015-07-10,36059247,Greg,The apartment was as advertised and Frank was ...,the apartment was as advertised and frank was ...
1,3781,41842494,2015-08-09,10459388,Tai,It was a pleasure to stay at Frank's place. Th...,it was a pleasure to stay at frank's place. th...
2,3781,45282151,2015-09-01,12264652,Damien,The apartment description is entirely faithful...,the apartment description is entirely faithful...
3,3781,49022647,2015-09-30,41426327,Mike,Thoroughly enjoyed my time at Frank's home. Ha...,thoroughly enjoyed my time at frank's home. ha...
4,3781,52503327,2015-10-30,15151513,Ivan,Great value for the money! This location has e...,great value for the money! this location has e...
